<a href="https://colab.research.google.com/github/saptarshidatta96/MTech_Sem3/blob/main/domain_shift_robustness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.12.0

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.12.0`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [2]:
import sys
print("User Current Version:-", sys.version)

User Current Version:- 3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]


In [3]:
import tensorflow
print(tensorflow.__version__)

1.15.2


In [4]:
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [5]:
from tensorflow.contrib import slim

In [6]:
import _pickle as cPickle

In [7]:
import numpy as np
import os
from PIL import Image
from tensorflow.examples.tutorials.mnist import input_data




def resize_images(image_arrays, size=[32,32]):
    image_arrays = (image_arrays * 255).astype('uint8')
    
    resized_image_arrays = np.zeros([image_arrays.shape[0]]+size)
    for i, image_array in enumerate(image_arrays):
        image = Image.fromarray(image_array)
        resized_image = image.resize(size=size, resample=Image.ANTIALIAS)
        
        resized_image_arrays[i] = np.asarray(resized_image)
    
    return np.expand_dims(resized_image_arrays, 3)  

def download_and_process_mnist():
    
    
    if not os.path.exists('./data/mnist'):
      os.makedirs('./data/mnist')
    
    mnist = input_data.read_data_sets(train_dir='./data/mnist')

    train = {'X': resize_images(mnist.train.images.reshape(-1, 28, 28)),
             'y': mnist.train.labels}
    
    test = {'X': resize_images(mnist.test.images.reshape(-1, 28, 28)),
            'y': mnist.test.labels}
        
    with open('./data/mnist/train.pkl','wb') as f:
      cPickle.dump(train,f)
    
    with open('./data/mnist/test.pkl','wb') as f:
      cPickle.dump(test,f)
    
if __name__ == "__main__":
    download_and_process_mnist()
    


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/mnist/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


search ops

In [8]:

import numpy as np
import numpy.random as npr
from configparser import *
import os

import scipy.io
import sys
import glob
from numpy.linalg import norm
from scipy import misc
import skimage.transform

import PIL.Image



class SearchOps(object):
	
	'''
	Class to handle all the search procedures.
	Currently implemented: random search and evolution search
	'''

	def __init__(self):
		self.transf_ops = TransfOps()

	def random_search(self, no_iters, string_length, save_file_name, compute_fitness_f, original_images, *args):

		'''
		Sampling random image transformations and testing them on a provided model.
		Referring to the paper, this is Algorithm 1.
		
			no_iters: number of iterations.
			string_length: number of transformations to be concatenated.
			save_file_name: file name used to save .png and .pkl outputs.
			compute_fitness_f: test function associated with the desired model.
			original_images: images to give in input to compute_fitness_f.
			args: other input eventually required by compute_fitness_f (e.g., ground truth labels, sess, etc.)

		'''

		all_accuracies = []
		all_best_accuracies = []
		all_transformations = []
		all_levels = []
		all_images = []
		current_minimum = 1.
		
		number_fitness_evals = 0		
		
		for t in range(no_iters):

			if (t%100)==0:
				print('Iter #',str(t))

			tr_images, transformations, levels = self.transf_ops.transform_dataset(original_images * 255., transf_string='random_'+str(string_length))
			
			tr_images /= 255.
			
			N = 1 #set accordingly to GPU memory
			target_accuracy = 0
			target_loss = 0

			number_fitness_evals += 1

			(target_accuracy, preds) = compute_fitness_f(tr_images, *args)

			all_accuracies.append(target_accuracy)

			if target_accuracy < current_minimum:
				print ('%d Current minimum: [%.4f], # fitness evals: [%d]'%(t, target_accuracy, number_fitness_evals))
				current_minimum=target_accuracy				
			
				all_best_accuracies.append(target_accuracy)
				all_transformations.append(transformations)
				all_levels.append(levels)

				for n, pred in enumerate(preds):
					tr_images[n][:5,:5,:] = 0.
					tr_images[n][:5,:5,1] = pred
					tr_images[n][:5,:5,0] = (1.-pred)
								
				conc_images=np.vstack((np.hstack((tr_images[i]*255. for i in range(j*20, (j+1)*20))) for j in range(10)))
				all_images.append(conc_images)


		print('Saving output in "images" folder')
		PIL.Image.fromarray(conc_images.astype('uint8')).save(save_file_name+'_acc_%.3f.png'%(current_minimum))
				
		with open(save_file_name+'_acc_%.3f.pkl'%(current_minimum), 'wb') as f:
			cPickle.dump((all_accuracies, all_best_accuracies, all_transformations, all_levels, number_fitness_evals), f, cPickle.HIGHEST_PROTOCOL)

		return all_best_accuracies[-1], all_transformations[-1].tolist(), all_levels[-1], all_images[-1]
			
	def genetic_algorithm(self, no_iters, pop_size, string_length, mutation_rate, save_file_name, compute_fitness_f, original_images, *args):

		'''
		Sampling random image transformations and testing them on a provided model.
		Referring to the paper, this is Algorithm 2.
		
			no_iters: number of iterations.
			string_length: number of transformations to be concatenated.
			mutation_rate: a value in [0.0,1.0]
			save_file_name: file name used to save .png and .pkl outputs.
			compute_fitness_f: test function associated with the desired model.
			original_images: images to give in input to compute_fitness_f.
			args: other input eventually required by compute_fitness_f (e.g., ground truth labels, sess, etc.)

		'''

		min_accuracy = 1.0 # initialized with the maximum value
		current_minimum = 1.0 # initialized with the maximum value

		number_fitness_evals = 0
		number_fitness_needed = pop_size

		pop_accuracies = []
		pop_probabilities = []
		pop_transformations = []
		pop_levels = []
		pop_images = []

		min_accs = []
		min_transfs = []
		min_levels = []
		min_images = []

		all_fitnesses = []

		print('Initializing population')
			
		for p in range(pop_size): # number of items in the population

			tr_images, transformations, levels = self.transf_ops.transform_dataset(original_images * 255., transf_string='random_'+str(string_length))
			tr_images /= 255.

			N = 1 #set accordingly to GPU memory
			target_accuracy = 0
			target_loss = 0

			number_fitness_evals += 1
			
			(target_accuracy, preds) = compute_fitness_f(tr_images, *args)

			pop_accuracies.append(target_accuracy)
			pop_transformations.append(transformations)
			pop_levels.append(levels)
			
			for n, pred in enumerate(preds):
				tr_images[n][:5,:5,:] = 0.
				tr_images[n][:5,:5,1] = pred
				tr_images[n][:5,:5,0] = (1.-pred)
			
			conc_images=np.vstack((np.hstack((tr_images[i]*255. for i in range(j*20, (j+1)*20))) for j in range(10)))
			pop_images.append(conc_images)
			
		pop_probabilities = (1. - np.array(pop_accuracies))/np.sum(1. - np.array(pop_accuracies)) 

		current_minimum = np.min(pop_accuracies)
		print('Current minimum:',str(current_minimum), '# fitness evals', str(number_fitness_evals))

		min_accs.append(current_minimum)

		all_fitnesses.append(current_minimum)
		
		pop_transformations = [arr.tolist() for arr in pop_transformations]			

		min_transfs.append(pop_transformations[np.argmin(pop_accuracies)])
		min_levels.append(pop_levels[np.argmin(pop_accuracies)])
		min_images.append(pop_images[np.argmin(pop_accuracies)])

		print('Running evolution search')

		for step in range(no_iters): # number of iters for the evolution search

			if current_minimum == 0.0:
				break

			new_pop_accuracies = []
			new_pop_images = []
			new_pop_transformations = [None for i in range(pop_size)]
			new_pop_levels = [None for i in range(pop_size)]

			for p in range(pop_size/2):
					
				# randomly choose two parents to be mated <3
					
				idx_1 = npr.choice(pop_size, p=pop_probabilities)
				idx_2 = npr.choice(pop_size, p=pop_probabilities)

				transformations_1 = pop_transformations[idx_1]
				transformations_2 = pop_transformations[idx_2]
				levels_1 = pop_levels[idx_1]
				levels_2 = pop_levels[idx_2]
				
				# cutting transformations/levels on a random point and 
					
				crossover_point = npr.randint(string_length)
					
				new_transformations_1 = transformations_1[:crossover_point] + transformations_2[crossover_point:]
				new_levels_1 = levels_1[:crossover_point] + levels_2[crossover_point:]
				new_transformations_2 = transformations_2[:crossover_point] + transformations_1[crossover_point:]
				new_levels_2 = levels_2[:crossover_point] + levels_1[crossover_point:]
				
				# adding the new offspring to the new population				

				new_pop_transformations[p] = new_transformations_1
				new_pop_levels[p] = new_levels_1
				new_pop_transformations[p+pop_size/2] = new_transformations_2
				new_pop_levels[p+pop_size/2] = new_levels_2
							
			# mutating some genes
				
			for i, transformations in enumerate(new_pop_transformations):
				for j, transf in enumerate(transformations):
					if npr.rand() < mutation_rate: 
						new_pop_transformations[i][j] = npr.choice(self.transf_ops.transformation_list, 1)[0]
						new_pop_levels[i][j] = npr.choice(self.transf_ops.code_to_level_dict[new_pop_transformations[i][j]].values(), 1)[0]
																
			# computing accuracies ("fitness" values)

			for transformations, levels in zip(new_pop_transformations, new_pop_levels): 

				tr_images, _, _ = self.transf_ops.transform_dataset(original_images * 255., transformations=transformations, levels=levels)
				tr_images /= 255.
				
				N = 1 #set accordingly to GPU memory
				target_accuracy = 0
				target_loss = 0
			
				number_fitness_evals += 1
			
				(target_accuracy, preds) = compute_fitness_f(tr_images, *args)
				
				new_pop_accuracies.append(target_accuracy)


				for n, pred in enumerate(preds):
					tr_images[n][:5,:5,:] = 0.
					tr_images[n][:5,:5,1] = pred
					tr_images[n][:5,:5,0] = (1.-pred)

				conc_images=np.vstack((np.hstack((tr_images[i]*255. for i in range(j*20, (j+1)*20))) for j in range(10)))
				new_pop_images.append(conc_images)
			
			
			pop_transformations = new_pop_transformations
			pop_levels = new_pop_levels
			pop_accuracies = new_pop_accuracies

			pop_images = new_pop_images
			pop_probabilities = (1. - np.array(pop_accuracies))/np.sum(1. - np.array(pop_accuracies)) 

			if np.min(pop_accuracies) < current_minimum:
				current_minimum = np.min(pop_accuracies)
				print(str(step), '- Current minimum:', str(current_minimum), '#number fitness evals', str(number_fitness_evals))
				print(pop_transformations[np.argmin(pop_accuracies)])
				print(pop_levels[np.argmin(pop_accuracies)])

				number_fitness_needed = number_fitness_evals

				min_accs.append(current_minimum)
				min_transfs.append(pop_transformations[np.argmin(pop_accuracies)])
				min_levels.append(pop_levels[np.argmin(pop_accuracies)])
				min_images.append(pop_images[np.argmin(pop_accuracies)])

				
			all_fitnesses.append(current_minimum)

		PIL.Image.fromarray(pop_images[np.argmin(pop_accuracies)].astype('uint8')).save(save_file_name+'_acc_%.3f.png'%(current_minimum))
		
		with open(save_file_name+'_acc_%.3f.pkl'%(current_minimum), 'wb') as f:
			cPickle.dump((min_accs,min_transfs, min_levels, number_fitness_needed, all_fitnesses), f, cPickle.HIGHEST_PROTOCOL)

		return min_accs[np.argmin(min_accs)], min_transfs[np.argmin(min_accs)], min_levels[np.argmin(min_accs)], min_images[np.argmin(min_accs)]

if __name__=='__main__':

    print('...')



...


Tranf ops

In [9]:
import os
import numpy as np
import numpy.random as npr
import PIL
import PIL.ImageOps
import PIL.ImageEnhance
import PIL.Image
import matplotlib

class TransfOps(object):

	'''
	Class to handle the decoding of the strings used with the genetic
	algorithm and all the data transformations.
	'''
	
	def __init__(self):
				
		self.transformation_list = ['autocontrast', 'brightness', 'color', 'contrast', 'sharpness', 'solarize', 'grayscale', 'Renhancer', 'Genhancer', 'Benhancer']
		self.define_code_correspondances()
		
	def decode_string(self, transf_string):
		
		'''
		Code to decode the string used by the genetic algorithm
		String example: 't1,l1_3,t4,l4_0,t0,l0_1'. First transformation is the one 
		associated with index '1', with level set to '3', and so on.
		'random_N' with N integer gives N rnd transformations with rnd levels.
		'''
		
		if 'random' in transf_string:
			transformations = npr.choice(self.transformation_list, int(transf_string.split('_')[-1])) # the string is 'random_N'
			levels = [npr.choice(list(self.code_to_level_dict[t].values()), 1)[0] for t in transformations] # list() to make it compatible with Python3
		else:
			transformation_codes = transf_string.split(',')[0::2] 
			level_codes = transf_string.split(',')[1::2]
			
			transformations = [self.code_to_transf(code) for code in transformation_codes] 	
			levels = [self.code_to_level(transf,level) for transf,level in zip(transformations, level_codes)] 	

		return transformations, levels		

	def transform_dataset(self, dataset, transf_string = 't0,l0_0', transformations=None, levels=None):
		
		'''
		dataset: set of images, shape should be N x width x height x #channels
		transf_string: transformations and levels encoded in a string 
		'''

		#print 'Dataset size:',dataset.shape
		
		if len(dataset.shape) == 3: # if 'dataset' is a single image
			dataset = np.expand_dims(dataset, 0) 
		
		if dataset.shape[-1] != 3:
			print('Input shape:', str(dataset.shape))
			raise Exception('The images must be in RGB format')

		tr_dataset = np.zeros((dataset.shape))
		
		if transformations is None:
			# decoding transformation string
			transformations, levels = self.decode_string(transf_string)	
		
		for n,img in enumerate(dataset):
			pil_img = PIL.Image.fromarray(img.astype('uint8'), 'RGB')
			for transf,level in zip(transformations, levels): 
				pil_img = self.apply_transformation(pil_img, transf, level)
			tr_dataset[n] = np.array(pil_img)

		return tr_dataset, transformations, levels

	def apply_transformation(self, image, transformation, level):
		
		'''
		image: image to be tranformed, shape should be 1 x width x height x #channels
		transformation: type of transformation to be applied
		level: level of the perturbation to be applied 
		'''

		if transformation == 'identity':
			return image 

		elif transformation == 'autocontrast':
			return PIL.ImageOps.autocontrast(image, cutoff=level)

		elif transformation == 'brightness':
			return PIL.ImageEnhance.Brightness(image).enhance(level)

		elif transformation == 'color':
			return PIL.ImageEnhance.Color(image).enhance(level)
			
		elif transformation == 'contrast':
			return PIL.ImageEnhance.Contrast(image).enhance(level)

		elif transformation == 'sharpness':
			return PIL.ImageEnhance.Sharpness(image).enhance(level)
			
		elif transformation == 'solarize':
			return PIL.ImageOps.solarize(image, threshold=level)

		elif transformation == 'grayscale':
			image = PIL.ImageOps.grayscale(image).convert('RGB')
			return image		

		elif transformation == 'Renhancer':
			image = np.array(image).astype(int)
			image[:,:,0] += level
			image[image>255] = 255
			image[image<0] = 0
			
			image = PIL.Image.fromarray(image.astype('uint8'), 'RGB')
			return image

		elif transformation == 'Genhancer':
			image = np.array(image).astype(int)
			image[:,:,1] += level
			image[image>255] = 255
			image[image<0] = 0
			image = PIL.Image.fromarray(image.astype('uint8'), 'RGB')
			return image

		elif transformation == 'Benhancer':
			image = np.array(image).astype(int)
			image[:,:,2] += level
			image[image>255] = 255
			image[image<0] = 0
			image = PIL.Image.fromarray(image.astype('uint8'), 'RGB')
			return image

	def code_to_transf(self, code):
		
		'''
		Takes in input a code (e.g., 't0', 't1', ...) and gives in output 
		the related transformation.
		'''

		return self.code_to_transf_dict[code]


	def code_to_level(self, transformation, code):
		
		'''
		Takes in input a transfotmation (e.g., 'invert', 'colorize', ...) and 
		a level code (e.g., 'l0_1', 'l1_3', ...) and gives in output the related level.
		'''
		
		return self.code_to_level_dict[transformation][code]
				
	def define_code_correspondances(self):
		
		'''
		Define the correpondances between transformation/level codes
		and the actual types and values.
		'''
			
		self.code_to_transf_dict = dict()
		
		self.code_to_transf_dict['t1'] = 'autocontrast'
		self.code_to_transf_dict['t2'] = 'brightness'
		self.code_to_transf_dict['t3'] = 'color'
		self.code_to_transf_dict['t4'] = 'contrast'
		self.code_to_transf_dict['t5'] = 'sharpness'
		self.code_to_transf_dict['t6'] = 'solarize'
		self.code_to_transf_dict['t7'] = 'grayscale'
		self.code_to_transf_dict['t8'] = 'Renhancer'
		self.code_to_transf_dict['t9'] = 'Genhancer'
		self.code_to_transf_dict['t10'] = 'Benhancer'

		self.code_to_level_dict = dict()
		
		for k in self.transformation_list:
			self.code_to_level_dict[k] = dict()
			
		# percentages
		self.code_to_level_dict['autocontrast'] = dict()
		for n,l in enumerate(np.linspace(0.0,0.3,20)):
			self.code_to_level_dict['autocontrast']['l1_'+str(n)] = l

		# factors
		self.code_to_level_dict['brightness'] = dict()
		for n,l in enumerate(np.linspace(0.6,1.4,20)):
			self.code_to_level_dict['brightness']['l2_'+str(n)] = l
		
		# factors
		self.code_to_level_dict['color'] = dict()
		for n,l in enumerate(np.linspace(0.6,1.4,20)):
			self.code_to_level_dict['color']['l3_'+str(n)] = l
		
		# factors
		self.code_to_level_dict['contrast'] = dict()
		for n,l in enumerate(np.linspace(0.6,1.4,20)):
			self.code_to_level_dict['contrast']['l4_'+str(n)] = l

		# factors
		self.code_to_level_dict['sharpness'] = dict()
		for n,l in enumerate(np.linspace(0.6,1.4,20)):
			self.code_to_level_dict['sharpness']['l5_'+str(n)] = l
		
		self.code_to_level_dict['solarize'] = dict()
		for n,l in enumerate(np.linspace(0,20,20).astype(int)):
			self.code_to_level_dict['solarize']['l6_'+str(n)] = l

		self.code_to_level_dict['grayscale']['l7_0'] = None

		# percentages
		self.code_to_level_dict['Renhancer'] = dict()
		for n,l in enumerate(np.linspace(-120,120,30).astype(int)):
			self.code_to_level_dict['Renhancer']['l8_'+str(n)] = l

		# percentages
		self.code_to_level_dict['Genhancer'] = dict()
		for n,l in enumerate(np.linspace(-120,120,30).astype(int)):
			self.code_to_level_dict['Genhancer']['l9_'+str(n)] = l

		# percentages
		self.code_to_level_dict['Benhancer'] = dict()
		for n,l in enumerate(np.linspace(-120,120,30).astype(int)):
			self.code_to_level_dict['Benhancer']['l10_'+str(n)] = l

if __name__=='__main__':
	pass









In [10]:

import numpy as np
import numpy.random as npr
import configparser
import os
#import cPickle
import scipy.io
import sys
import glob
from numpy.linalg import norm
from scipy import misc
import skimage.transform

import PIL.Image

sys.path.insert(0,'../')


class TrainOps(object):

	def __init__(self, model, exp_dir):

		self.model = model
		self.exp_dir = exp_dir

		self.config = tf.ConfigProto()
		self.config.gpu_options.allow_growth=False

		self.data_dir = './data'

	def load_exp_config(self):

		print(self.exp_dir)

		config = configparser.ConfigParser()

		print('LOADING CONFIG FILE')
		config.read(os.path.join(self.exp_dir,'exp_config'))
		self.source_dataset = config.get('EXPERIMENT_SETTINGS', 'source_dataset')

		self.model.source_dataset = self.source_dataset
			
		self.model.no_classes = 10
		self.model.img_size = 32

		self.log_dir = os.path.join(self.exp_dir,'logs')
		self.model_save_path = os.path.join(self.exp_dir,'model')
		self.images_dir = os.path.join(self.exp_dir,'images')

		if not os.path.exists(self.log_dir):
			os.makedirs(self.log_dir)

		if not os.path.exists(self.model_save_path):
			os.makedirs(self.model_save_path)

		if not os.path.exists(os.path.join(self.images_dir)):
			os.makedirs(os.path.join(self.images_dir))


		self.train_iters = config.getint('MAIN_SETTINGS', 'train_iters')
		self.batch_size = config.getint('MAIN_SETTINGS', 'batch_size')
		self.model.batch_size = self.batch_size
		self.model.learning_rate = config.getfloat('MAIN_SETTINGS', 'learning_rate')

		self.transf_string = config.get('MAIN_SETTINGS', 'transf_string')
		self.sub_train_iters = config.getint('MAIN_SETTINGS', 'sub_train_iters')
		self.string_length = config.getint('MAIN_SETTINGS', 'string_length')

		self.transf_ops = TransfOps()
		self.search_ops = SearchOps()

	def load_svhn(self, split='train'):

		print ('Loading SVHN dataset.')

		image_file = 'train_32x32.mat' if split=='train' else 'test_32x32.mat'

		image_dir = os.path.join(self.data_dir, 'svhn', image_file)
		svhn = scipy.io.loadmat(image_dir)
		images = np.transpose(svhn['X'], [3, 0, 1, 2])
		labels = svhn['y'].reshape(-1)
		labels[np.where(labels==10)] = 0
		images = images/255.
		return images, labels

	def load_mnist(self, split='train'):

		print ('Loading MNIST dataset.')
		image_file = 'train.pkl' if split=='train' else 'test.pkl'
		image_dir = os.path.join(self.data_dir, 'mnist', image_file)
		with open(image_dir, 'rb') as f:
			mnist = cPickle.load(f)
		
		images = mnist['X'] 
		labels = mnist['y']

		images = images
		images = images/255. # better generalization performance if [0,1]

		images = np.stack((images,images,images), axis=3) # grayscale to rgb

		return np.squeeze(images), labels

	def load_mnist_m(self, split='train'):

		print ('Loading MNIST_M dataset.')


		image_dir = os.path.join(self.data_dir,'mnist_m')

		if split == 'train':
			data_dir = os.path.join(image_dir,'mnist_m_train')
			with open(os.path.join(image_dir,'mnist_m_train_labels.txt')) as f:
				content = f.readlines()
				
		elif split == 'test':
			data_dir = os.path.join(image_dir,'mnist_m_test')
			with open(os.path.join(image_dir,'mnist_m_test_labels.txt')) as f:
				content = f.readlines()


		content = [c.split('\n')[0] for c in content]
		images_files = [c.split(' ')[0] for c in content]
		labels = np.array([int(c.split(' ')[1]) for c in content]).reshape(-1)

		images = np.zeros((len(labels), 32, 32, 3))

		for no_img,img in enumerate(images_files):
			img_dir = os.path.join(data_dir, img)
			im = misc.imread(img_dir)
			im = np.expand_dims(im, axis=0)
			images[no_img] = im

		images = images 
		images = images/255.
		
		return images, labels

	def load_syn(self, split='train'):
		print ('Loading SYN dataset.')

		image_file = 'synth_train_32x32.mat' if split=='train' else 'synth_test_32x32.mat'

		image_dir = os.path.join(self.data_dir,'syn', image_file)
		syn = scipy.io.loadmat(image_dir)
		images = np.transpose(syn['X'], [3, 0, 1, 2])
		labels = syn['y'].reshape(-1)
		labels[np.where(labels==10)] = 0
		
		images = images/255.
		return images, labels

	def load_usps(self, split='train'):

		print ('Loading USPS dataset.')
		image_file = 'usps_32x32.pkl'
		image_dir = os.path.join(self.data_dir,'usps', image_file)
		
		with open(image_dir, 'rb') as f:
			usps = cPickle.load(f)
		
		images = usps['X']
		labels = usps['y']
		labels -= 1
		labels[labels==255] = 9

		images=np.squeeze(images)
		images = np.stack((images,images,images), axis=3) # grayscale to rgb
		images = images/255.

		if split == 'train':
			return images[:6562], np.squeeze(labels[:6562]).astype(int)
		elif split == 'validation':
			return images[6562:7291], np.squeeze(labels[6562:7291]).astype(int)
		elif split == 'test':	    
			return images[7291:], np.squeeze(labels[7291:]).astype(int)
	
	def load_test_data(self, target):

		if target=='mnist_m':
			self.target_test_images, self.target_test_labels = self.load_mnist_m(split='test')
		elif target=='svhn':
			self.target_test_images, self.target_test_labels = self.load_svhn(split='test')
		elif target=='syn':
			self.target_test_images, self.target_test_labels = self.load_syn(split='test')
		elif target=='usps':
			self.target_test_images, self.target_test_labels = self.load_usps(split='test')
		elif target=='mnist':
			self.target_test_images, self.target_test_labels = self.load_mnist(split='test')

		return self.target_test_images,self.target_test_labels

	def train(self, random_transf=False): 

		'''
		This method allows to train ERM and RDA models.
		
			random_transf: if set to True, RDA is used, o.w. ERM.
		
		The number of transformations to be concatenated needs be to
		set in the file exp_config.
		'''

		# build a graph
		print('Building model')
		self.model.build_model()
		print('Built')

		print('Loading data')

		source_train_images, source_train_labels = self.load_mnist(split='train')
		target_test_images, target_test_labels = self.load_mnist(split='test')

		with tf.Session(config=self.config) as sess:
			tf.global_variables_initializer().run()

			saver = tf.train.Saver()

			summary_writer = tf.summary.FileWriter(logdir=self.log_dir, graph=tf.get_default_graph())

			print('Training')
			
			for t in range(self.train_iters):

				i = t % int(source_train_images.shape[0] / self.batch_size)

				#current batch of images and labels
				batch_images = source_train_images[i*self.batch_size:(i+1)*self.batch_size]
				batch_labels = source_train_labels[i*self.batch_size:(i+1)*self.batch_size]

				if random_transf:
					batch_images, _, _ = self.transf_ops.transform_dataset(batch_images * 255., transf_string = self.transf_string)
					batch_images /= 255.
				
				feed_dict = {self.model.images: batch_images, self.model.labels: batch_labels} 

				#running a step of gradient descent
				sess.run([self.model.min_train_op, self.model.min_loss], feed_dict) 

				#evaluating the model
				if t % 2500 == 0:

					summary, min_l, acc = sess.run([self.model.summary_op, self.model.min_loss, self.model.accuracy], feed_dict)

					train_rand_idxs = np.random.permutation(source_train_images.shape[0])[:100]
					test_rand_idxs = np.random.permutation(target_test_images.shape[0])[:100]

					train_acc, train_min_loss = sess.run(fetches=[self.model.accuracy, self.model.min_loss], 
					feed_dict={self.model.images: source_train_images[train_rand_idxs], 
					self.model.labels: source_train_labels[train_rand_idxs]})
					
					test_acc, test_min_loss = sess.run(fetches=[self.model.accuracy, self.model.min_loss], 
					feed_dict={self.model.images: target_test_images[test_rand_idxs], 
					self.model.labels: target_test_labels[test_rand_idxs]})
					  
					summary_writer.add_summary(summary, t)
					print ('Step: [%d/%d] train_min_loss: [%.4f] train_acc: [%.4f] test_min_loss: [%.4f] test_acc: [%.4f]'%(t+1, self.train_iters, train_min_loss, train_acc, test_min_loss, test_acc))
			
				if t % 10000 == 0:
					print('Saving')
					saver.save(sess, os.path.join(self.model_save_path, 'encoder'))

	def train_search(self, search_algorithm='random_search'): 
		
		'''
		This method allows to train models using RSDA and ESDA algorithms.
		Referring to the paper, this is Algorithm 3.

			search_algorithm: 'random_search' or 'evolution_search', 
							  accordingly to the desired search procedure.

		The number of transformations to be concatenated needs be to
		set in the file exp_config.
		'''

		# build a graph
		print('Building model')
		self.model.build_model()
		print('Built')

		print('Loading data')

		source_train_images, source_train_labels = self.load_mnist(split='train')
		target_test_images, target_test_labels = self.load_mnist(split='test')

		# initializing the set of data augmentation rules.

		transformations = [['identity']]
		levels = [[None]]		

		with tf.Session(config=self.config) as sess:

			tf.global_variables_initializer().run()

			saver = tf.train.Saver()

			summary_writer = tf.summary.FileWriter(logdir=self.log_dir, graph=tf.get_default_graph())

			print('Training')
			
			for t in range(self.train_iters):

				i = t % int(source_train_images.shape[0] / self.batch_size)

				# current batch of images and labels
				batch_images = source_train_images[i*self.batch_size:(i+1)*self.batch_size]
				batch_labels = source_train_labels[i*self.batch_size:(i+1)*self.batch_size]

				# sampling uniformly a transformation and its level, and applying it to the batch
				rnd_transf_idx = npr.randint(len(transformations))
				
				if transformations[rnd_transf_idx] == ['identity']: # do nothing for 'identity', namely use original images
					pass
				else:
					# TransfOps requires [0,255] pixel ranges, while here images [0,1]
					batch_images, _, _ = self.transf_ops.transform_dataset(batch_images * 255., transformations=transformations[rnd_transf_idx], levels=levels[rnd_transf_idx])
					batch_images /= 255.
								
				# running a step of gradient descent
				feed_dict = {self.model.images: batch_images, self.model.labels: batch_labels} 
				sess.run([self.model.min_train_op, self.model.min_loss], feed_dict) 

				#evaluating the model
				if t % 2500 == 0:

					summary, min_l, acc = sess.run([self.model.summary_op, self.model.min_loss, self.model.accuracy], feed_dict)

					train_rand_idxs = np.random.permutation(source_train_images.shape[0])[:100]
					test_rand_idxs = np.random.permutation(target_test_images.shape[0])[:100]

					train_acc, train_min_loss = sess.run(fetches=[self.model.accuracy, self.model.min_loss], 
					feed_dict={self.model.images: source_train_images[train_rand_idxs], 
					self.model.labels: source_train_labels[train_rand_idxs]})
					test_acc, test_min_loss = sess.run(fetches=[self.model.accuracy, self.model.min_loss], 
					feed_dict={self.model.images: target_test_images[test_rand_idxs], 
					self.model.labels: target_test_labels[test_rand_idxs]})
					  
					summary_writer.add_summary(summary, t)
					print('Step: [%d/%d] train_min_loss: [%.4f] train_acc: [%.4f] test_min_loss: [%.4f] test_acc: [%.4f]'%(t+1, self.train_iters, train_min_loss, train_acc, test_min_loss, test_acc))

				if (t+1)%self.sub_train_iters == 0:
					
					if search_algorithm == 'random_search':						
						print('\n\nRunning Random Search')
						save_file_name=os.path.join(self.images_dir,'Random_string_length_'+str(self.string_length)+'_iter_'+str(t+1))
						min_tr_accuracy, _transformations, _levels, _image = self.search_ops.random_search(100, self.string_length, save_file_name,
																										self.test, source_train_images[:1000],
																										source_train_labels[:1000], sess) 
					
					elif search_algorithm == 'evolution_search':						
						print('\n\nRunning Evolution Search')
						save_file_name=os.path.join(self.images_dir,'Evolution_string_length_'+str(self.string_length)+'_iter_'+str(t+1))
						min_tr_accuracy, _transformations, _levels, _image = self.search_ops.genetic_algorithm(10, 10, self.string_length, 0.1,
																											save_file_name, self.test, 
																											source_train_images[:1000],	source_train_labels[:1000], sess) 
																									
					transformations.append(_transformations)
					levels.append(_levels)

					print('Target accuracy: [%.4f]'%(min_tr_accuracy))
					print('_'.join(_transformations))
					print('\n\n')		

				if (t+1) % 25000 == 0:
					print('Saving')
					saver.save(sess, os.path.join(self.model_save_path, 'encoder'))


	def test(self, images, labels, sess):

		N = 1 #set accordingly to GPU memory
		target_accuracy = 0
		target_loss = 0
		preds = []

		for test_images_batch, test_labels_batch in zip(np.array_split(images, N), np.array_split(labels, N)):
			feed_dict = {self.model.images: test_images_batch, self.model.labels: test_labels_batch} 
			target_accuracy_tmp, target_loss_tmp, pred = sess.run([self.model.accuracy, self.model.min_loss, self.model.pred], feed_dict) 
			target_accuracy += target_accuracy_tmp/float(N)
			target_loss += target_loss_tmp/float(N)
			preds.append(pred.tolist())

		correct_guesses = (np.array(preds)==labels).astype(int)[0]
		
		return target_accuracy, correct_guesses
	
	def test_all(self):

		# build a graph
		print('Building model')
		self.model.build_model()
		print('Built')

		res_dict = dict()
		res_dict['exp_dir'] = self.exp_dir

		print('Testing ALL')

		targets = ['mnist', 'svhn']# add 'usps', 'syn', 'mnist_m'

		for target in targets:

			print('\n\n\n...........................................................................')

			test_images, test_labels = self.load_test_data(target=target)

			with tf.Session() as sess:

				print('...........................................................................')

				tf.global_variables_initializer().run()

				
				print('Loading pre-trained model.')
				variables_to_restore = slim.get_model_variables()
				restorer = tf.train.Saver(variables_to_restore)
				restorer.restore(sess, os.path.join(self.model_save_path,'encoder'))
				
				N = 100
				target_accuracy = 0
				target_loss = 0

				print('Calculating accuracy')

				for test_images_batch, test_labels_batch in zip(np.array_split(test_images, N), np.array_split(test_labels, N)):
					feed_dict = {self.model.images: test_images_batch, self.model.labels: test_labels_batch} 
					target_accuracy_tmp, target_loss_tmp, target_pred = sess.run([self.model.accuracy, self.model.min_loss, self.model.pred], feed_dict) 
					target_accuracy += target_accuracy_tmp/float(N)
					target_loss += target_loss_tmp/float(N)

			print('Target accuracy: [%.4f] target loss: [%.4f]'%(target_accuracy, target_loss))

			res_dict[target] = target_accuracy

		with open(os.path.join(self.exp_dir, 'domain_generalization_performance.pkl'), 'w') as f:
			cPickle.dump(res_dict, f, cPickle.HIGHEST_PROTOCOL)


	def test_random_search(self, run, seed, no_iters, string_length):

		test_images, test_labels = self.load_test_data(target='mnist')
		
		npr.seed(213)
		rnd_idx = range(len(test_images))
		npr.shuffle(rnd_idx)		

		test_images = test_images[rnd_idx]
		test_labels = test_labels[rnd_idx]

		test_images = test_images[:1000]
		test_labels = test_labels[:1000]

		npr.seed(seed)

		# build a graph
		print('Building model')
		self.model.mode='train_encoder'
		self.model.build_model()
		print('Built')

		with tf.Session() as sess:

			tf.global_variables_initializer().run()

			print ('Loading pre-trained model.')
			variables_to_restore = slim.get_model_variables(scope='encoder')
			restorer = tf.train.Saver(variables_to_restore)
			restorer.restore(sess, os.path.join(self.model_save_path,'encoder'))

			if not os.path.exists(os.path.join(self.exp_dir,'images')):
				os.makedirs(os.path.join(self.exp_dir,'images'))

			# perform random search
			
			save_search_file_name=os.path.join(self.images_dir,'TEST_Random_test_string_length_'+str(string_length))

			all_accuracies, all_transformations, all_levels, all_images = self.search_ops.random_search(5000, string_length, save_search_file_name,	self.test, test_images, test_labels, sess) 
			# save output

			with open(os.path.join(self.exp_dir, 'worst_case_accuracies.pkl'), 'w') as f:
				cPickle.dump((all_accuracies, all_transformations, all_levels), f, cPickle.HIGHEST_PROTOCOL)

	def test_evolution_search(self, run='0', seed=123, no_iters=100, string_length=3, pop_size=10, mutation_rate=0.1):

		test_images, test_labels = self.load_mnist(split='test')
		
		npr.seed(213)
		rnd_idx = range(len(test_images))
		npr.shuffle(rnd_idx)		

		test_images = test_images[rnd_idx]
		test_labels = test_labels[rnd_idx]

		test_images = test_images[:1000]
		test_labels = test_labels[:1000]

		npr.seed(seed)

		# build a graph
		print('Building model')
		self.model.mode='train_encoder'
		self.model.build_model()
		print('Built')

		with tf.Session() as sess:

			tf.global_variables_initializer().run()

			print ('Loading pre-trained model.')
			variables_to_restore = slim.get_model_variables()
			restorer = tf.train.Saver(variables_to_restore)
			restorer.restore(sess, os.path.join(self.model_save_path,'encoder'))

			if not os.path.exists(os.path.join(self.exp_dir,'GA_images')):
				os.makedirs(os.path.join(self.exp_dir,'GA_images'))
						
			save_search_file_name=os.path.join(self.images_dir,'TEST_Evolution_test_string_length_'+str(string_length))

			self.search_ops.genetic_algorithm(100, pop_size, string_length, mutation_rate, save_search_file_name, self.test, test_images, test_labels, sess)

if __name__=='__main__':

    print('...')



...


In [11]:



class Model(object):
    
    def __init__(self, mode='train'):

        self.no_classes = 10
        self.img_size = 32
        self.no_channels = 3

    def encoder(self, images, reuse=False):

        with tf.variable_scope('encoder', reuse=reuse):
            with slim.arg_scope([slim.fully_connected], activation_fn=tf.nn.relu):
                with slim.arg_scope([slim.conv2d], activation_fn=tf.nn.relu, padding='VALID'):

                    net = slim.conv2d(images, 64, 5, scope='conv1')
                    net = slim.max_pool2d(net, 2, stride=2, scope='pool1')
                    net = slim.conv2d(net, 128, 5, scope='conv2')
                    net = slim.max_pool2d(net, 2, stride=2, scope='pool2')
                    net = tf.layers.flatten(net)
                    net = slim.fully_connected(net, 1024, scope='fc1')
                    net = slim.fully_connected(net, 1024, scope='fc2')
                    net = slim.fully_connected(net, self.no_classes, activation_fn=None, scope='fco')
                    
                    return net

    def build_model(self):

        # images placeholder
        self.images = tf.placeholder(tf.float32, [None, self.img_size, self.img_size, self.no_channels], 'images')
        # labels placeholder
        self.labels = tf.placeholder(tf.int64, [None], 'labels')
                
        self.logits = tf.squeeze(self.encoder(self.images))

        #for evaluation
        self.pred = tf.argmax(self.logits, 1)
        self.correct_pred = tf.equal(self.pred, self.labels)
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_pred, tf.float32))

        #variables for the minimizer are the net weights, variables for the maxmizer are the images' pixels
        min_vars = tf.trainable_variables()
                
        #loss for the minimizer
        self.min_loss = slim.losses.sparse_softmax_cross_entropy(self.logits, self.labels)

        #we use Adam for the minimizer and vanilla gradient ascent for the maximizer 
        self.min_optimizer = tf.train.AdamOptimizer(self.learning_rate) 

        #minimizer
        self.min_train_op = slim.learning.create_train_op(self.min_loss, self.min_optimizer, variables_to_train = min_vars)

        min_loss_summary = tf.summary.scalar('min_loss', self.min_loss)

        accuracy_summary = tf.summary.scalar('accuracy', self.accuracy)
        self.summary_op = tf.summary.merge([min_loss_summary, accuracy_summary])		




train.py

In [12]:
gpu = 0
exp_dir = '/content/'
mode = 'train_ERM'
run = 0
seed = 123
transf_string_length = 5
search_no_iters = 100
pop_size = 10
mutation_rate = 0.1

In [ ]:



import glob
import os


import numpy.random as npr
import numpy as np
import itertools


def main(_):


	GPU_ID = gpu
	os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152 on stackoverflow
	os.environ["CUDA_VISIBLE_DEVICES"] = str(GPU_ID)

	EXP_DIR = exp_dir

	model = Model()
	tr_ops = TrainOps(model, EXP_DIR)

	if 'train' in mode:
		npr.seed(int(seed))

	
	if mode=='train_ERM':
		print('Training model with standard ERM')
		tr_ops.load_exp_config()
		tr_ops.train()      
		
	if mode=='train_RDA':
		print('Training model with RDA')
		tr_ops.load_exp_config()
		tr_ops.train(random_transf=True)
		
	if mode=='train_RSDA':
		print('Training model with RSDA')
		tr_ops.load_exp_config()
		tr_ops.train_search(search_algorithm='random_search')
		
	if mode=='train_ESDA':
		print('Training model with ESDA')
		tr_ops.load_exp_config()
		tr_ops.train_search(search_algorithm='evolution_search')

		
	elif mode=='test_all':
		print('Testing all')
		tr_ops.load_exp_config()
		tr_ops.test_all()

	elif mode=='test_RS':
		print('Random search')
		tr_ops.load_exp_config()
		tr_ops.test_random_search(run=str(run), seed=int(seed), no_iters=int(search_no_iters), string_length=int(transf_string_length)) 

	elif mode=='test_ES':
		print('Evolution search')
		tr_ops.load_exp_config()
		tr_ops.test_evolution_search(run=str(run), seed=int(seed), no_iters=int(search_no_iters),string_length=int(transf_string_length), 
										pop_size=int(pop_size), mutation_rate=float(mutation_rate))

if __name__ == '__main__':
	tf.app.run()


Training model with standard ERM
/content/
LOADING CONFIG FILE
Building model
Instructions for updating:
Please use `layer.__call__` method instead.


W0430 07:05:15.196529 140221453285248 deprecation.py:323] From /tensorflow-1.15.2/python3.7/tensorflow_core/contrib/layers/python/layers/layers.py:1057: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.


Instructions for updating:
Use keras.layers.flatten instead.


W0430 07:05:15.268467 140221453285248 deprecation.py:323] From <ipython-input-11-905eaf5cb5a2>:22: flatten (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.flatten instead.


Instructions for updating:
Use tf.losses.sparse_softmax_cross_entropy instead. Note that the order of the logits and labels arguments has been changed.


...

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


W0430 07:05:15.537699 140221453285248 deprecation.py:323] From /tensorflow-1.15.2/python3.7/tensorflow_core/contrib/losses/python/losses/loss_ops.py:154: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.losses.add_loss instead.


W0430 07:05:15.557658 140221453285248 deprecation.py:323] From /tensorflow-1.15.2/python3.7/tensorflow_core/contrib/losses/python/losses/loss_ops.py:121: add_loss (from tensorflow.contrib.losses.python.losses.loss_ops) is deprecated and will be removed after 2016-12-30.
Instructions for updating:
Use tf.losses.add_loss instead.


Built
Loading data
Loading MNIST dataset.
Loading MNIST dataset.
Training
Step: [1/100001] train_min_loss: [2.2514] train_acc: [0.2200] test_min_loss: [2.2590] test_acc: [0.2600]
Saving
Step: [2501/100001] train_min_loss: [0.0282] train_acc: [0.9800] test_min_loss: [0.0143] test_acc: [1.0000]
Step: [5001/100001] train_min_loss: [0.0022] train_acc: [1.0000] test_min_loss: [0.0674] test_acc: [0.9600]
Step: [7501/100001] train_min_loss: [0.0541] train_acc: [0.9600] test_min_loss: [0.0240] test_acc: [0.9900]
Step: [10001/100001] train_min_loss: [0.0012] train_acc: [1.0000] test_min_loss: [0.0850] test_acc: [0.9900]
Saving
Step: [12501/100001] train_min_loss: [0.0007] train_acc: [1.0000] test_min_loss: [0.0557] test_acc: [0.9900]
Step: [15001/100001] train_min_loss: [0.0059] train_acc: [1.0000] test_min_loss: [0.0001] test_acc: [1.0000]
Step: [17501/100001] train_min_loss: [0.0010] train_acc: [1.0000] test_min_loss: [0.0351] test_acc: [0.9900]
Step: [20001/100001] train_min_loss: [0.0018] t

In [ ]:
%tb